## OWNER

Name: Flomics Biotech SL 

date: 19/06/2025

## MAIN

Generate gene coverage plot

## LIBRARIES

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import json

## FUNCTIONS/PAR_SETTING

### Set font for plots

In [ ]:
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Arial'
plt.rcParams['mathtext.it'] = 'Arial:italic'
plt.rcParams['mathtext.bf'] = 'Arial:bold'

### External scripts

In [ ]:
#dodged barplot function
script_dir = "./"
sys.path.append(script_dir)

from ridgeline_from_known_density_plot import ridgeline_from_known_density_plot

In [ ]:
def find_results_folder(base_dataset_path, prefix):
    for folder in os.listdir(base_dataset_path):
        if os.path.isdir(os.path.join(base_dataset_path, folder)):
            if prefix in folder and 'results' in folder:
                return folder
    return None

## WORKFLOW

### Setup

In [ ]:
base_folder = './'
chalasani_folder = './'
output_folder = os.getcwd()

datasets = {
    'block':         {'sliced': True,  'n_parts': 4,  'prefix': 'block'},
    'chalasani': {'sliced': True,  'n_parts': 6,  'prefix': 'chalasani'},
    'chen':          {'sliced': True,  'n_parts': 21, 'prefix': 'chen'},
    'flomics_2': {'sliced': False, 'subfolder_path': 'Flomics_2'},
    'decru':         {'sliced': True,  'n_parts': 6,  'prefix': 'decru'},
    'giraldez':      {'sliced': False,                 'prefix': 'giraldez_results_full'},
    'ibarra':        {'sliced': True,  'n_parts': 14, 'prefix': 'ibarra'},
    'moufarrej':     {'sliced': True,  'n_parts': 17, 'prefix': 'moufarrej'},
    'ngo':           {'sliced': False,                 'prefix': 'ngo'},
    'reggiardo':     {'sliced': False,                 'prefix': 'reggiardo'},
    'roskams':       {'sliced': True,  'n_parts': 6,  'prefix': 'roskams'},
    'rozowsky':      {'sliced': True,  'n_parts': 9,  'prefix': 'rozowsky'},
    'sun':           {'sliced': True,  'n_parts': 3,  'prefix': 'sun'},
    'tao':           {'sliced': True,  'n_parts': 12, 'prefix': 'tao'},
    #'taowei':        {'sliced': True,  'n_parts': 5,  'prefix': 'taowei'},
    'toden':         {'sliced': True,  'n_parts': 5, 'prefix': 'toden'},
    'wang':          {'sliced': True,  'n_parts': 4,  'prefix': 'wang'},
    'zhu':           {'sliced': True,  'n_parts': 5,  'prefix': 'zhu'},
}

dataset_colors = {
    "block": "#b3b3b3", "decru": "#009E73", "zhu": "#ffd633", "chen": "#997a00", "flomics_2": "#144D6B",
    "ngo": "#fa8072", "roskams": "#944dff", "moufarrej": "#CC79A7", "sun": "#D55E00",
    "tao": "#0072B2", "toden": "#800099", "ibarra": "#800000", "chalasani_merged": "#800040",
    "rozowsky": "#006600", "taowei": "#B32400", "giraldez": "#B1CC71", "reggiardo": "#F1085C", "wang": "#FE8F42"
}

### Save per sample gene coverage

In [ ]:
# ---- Process ----
all_profiles = {}

for dataset, meta in datasets.items():
    sliced = meta['sliced']
    n_parts = meta.get('n_parts', 1)
    prefix = meta.get('prefix', None)  # Only exists if sliced or traditional unsliced
    print(f"\nProcessing dataset: {dataset}")

    if sliced:
        for i in range(n_parts):
            print(f"\nProcessing dataset: {dataset+"_"+str(i)}")
            part_folder = f"{prefix}_results_{i:02d}.part/multiqc/star_salmon/multiqc_data"
            if dataset == "chalasani":
                part_folder = chalasani_folder+f"{prefix}_results_{i:02d}.part/multiqc/star_salmon/multiqc_data"
            file_path = os.path.join(base_folder, dataset, part_folder, "mqc_qualimap_gene_coverage_profile_Normalised.txt")
            if os.path.isfile(file_path):
                try:
                    df = pd.read_csv(file_path, sep="\t")
                    print(df.shape)
                    all_profiles[dataset+"_"+str(i)] = df
                    #handle special case (decru)
                    if dataset == "decru":
                        df["Sample"] = df["Sample"].str.split('_').str[0]
                    #handle special case (chalasani)
                    if dataset == "chalasani_merged":
                        df["Sample"] = "X" + df["Sample"].astype(str)
                    if dataset in ["chalasani_merged","decru","toden"]:
                        print(df.head())
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
            else:
                print(f"Missing: {file_path}")
    else:
        if "subfolder_path" in meta:
            part_path = os.path.join(base_folder, meta["subfolder_path"], "multiqc", "star_salmon", "multiqc_data")
        else:
            base_dataset_path = os.path.join(base_folder, dataset)
            matched_folder = find_results_folder(base_dataset_path, prefix)
            if not matched_folder:
                print(f"Could not find results folder for: {dataset}")
                continue
            part_path = os.path.join(base_dataset_path, matched_folder, "multiqc", "star_salmon", "multiqc_data")

        file_path = os.path.join(part_path, "mqc_qualimap_gene_coverage_profile_Normalised.txt")
        if os.path.isfile(file_path):
            try:
                df = pd.read_csv(file_path, sep="\t")
                print(df.shape)
                print(df.head())
                all_profiles[dataset+"_"+str(part_path)] = df
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
        else:
            print(f"Missing: {file_path}")

### preparing dataset for ridge plot

#### Merge gene coverage in one dataset

In [ ]:
# List to hold individual DataFrames
all_dataframes = []

# Iterate through the dictionary and append each DataFrame to the list
for key, df in all_profiles.items():
    all_dataframes.append(df)
    

# Concatenate all DataFrames into a single, unique dataset
sample_gene_coverage = pd.concat(all_dataframes)
sample_gene_coverage = sample_gene_coverage.set_index('Sample')
print(sample_gene_coverage.shape)
sample_gene_coverage.head()

#### Merge gene coverage with metadata (handle specific sample names)

In [ ]:
#load metadata
metadata = pd.read_csv("../tables/cfRNA-meta_per_sample_metadata.tsv",sep="\t")
#handle special case (Flomics_1 and Flomics2)
condition = metadata['dataset_batch'].str.contains(r'^flomics_.*', na=False)
metadata.loc[condition, 'run'] = metadata['sample_display_name']
#set index
metadata = metadata.set_index("run")
print(metadata.shape)
metadata.head()

In [ ]:
#merge with metadata
full_gene_coverage_data = sample_gene_coverage.join(metadata, how='inner')
print(full_gene_coverage_data.shape)

In [ ]:
list(set(sample_gene_coverage.index) - set(full_gene_coverage_data.index))

In [ ]:
list(set(metadata["dataset_short_name"].unique()) - set(full_gene_coverage_data["dataset_short_name"].unique()))

#### rename batches name with standard names and load batches proper colors

In [ ]:
#load JSON file
with open("./dataset_mappings.json") as json_file:
    data_mapping = json.load(json_file)

order_label = data_mapping['datasetVisualOrder']
color_dataset = data_mapping['datasetsPalette']
data_mapping = data_mapping['datasetsLabels']
order_label = [data_mapping.get(item, item) for item in order_label]

In [ ]:
#replace names in batches column
full_gene_coverage_data["dataset_batch"].replace(data_mapping, inplace=True)

In [ ]:
full_gene_coverage_data["dataset_batch"]

#### melt dataset

In [ ]:
#reset index
full_gene_coverage_data['Sample'] = full_gene_coverage_data.index
full_gene_coverage_data = full_gene_coverage_data.reset_index()

In [ ]:
position_list = [str(i) for i in range(0, 99)]
print(position_list)
print(full_gene_coverage_data.columns)
melt_dataset = full_gene_coverage_data.melt(id_vars=['dataset_batch'], value_vars=position_list)
melt_dataset["variable"] = pd.to_numeric(melt_dataset["variable"])
print(melt_dataset.shape)
melt_dataset.head()

In [ ]:
#correct dataset labels in color dict
dataset_batch_colors = {data_mapping.get(old_key, old_key): value 
            for old_key, value in color_dataset.items()}

print(dataset_batch_colors)

### ridge plot

#### default plot

In [ ]:
reload(heatmap_plot)

ridgeline_from_known_density_plot(
    dataset = melt_dataset,
    x_column = "variable",
    density_column = "value",
    category_column = "dataset_batch",
    fig_width = 10,
    fig_height = 10,
    output = "gene_coverage_merged",
    show_ylabel = False,
    xlabel = "Transcript position (%)",
    xlabel_va = 0.51,
    fill_color = dataset_batch_colors,
    show_title = False,
    line_width = 0.5,
    normalization = 'mean'
)

#### Y-axis plot

In [ ]:
reload(heatmap_plot)

ridgeline_from_known_density_plot(
    dataset = melt_dataset,
    x_column = "variable",
    density_column = "value",
    category_column = "dataset_batch",
    category_order = order_label[:-1],
    fig_width = 10,
    fig_height = 15,
    output = "gene_coverage_merged_y_axis",
    #layout
    show_ylabel = True,
    xlabel = "Transcript position (%)",
    xlabel_va = 0.51,
    fill_color = dataset_batch_colors,
    show_title = False,
    line_width = 0.5,
    y_ticks_frequency = 1,
    normalization = 'mean',
    #individual sub-plots
    show_individual_yaxis = True,
    show_individual_hspace = 0.1,
    consistent_y_scale = False,
    show_yticks = False,
    ylabel_fontsize = 10
)